February 2025
# MuTrans script: untreated samples

In [2]:
import sys
import os

print(f"Current directory: {os.getcwd()}")
#os.chdir("Z:\echen\MuTrans-release-main\Example")
#print(f"Changed directory: {os.getcwd()}")

import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc
import seaborn as sns
import hdf5plugin
import gc

import matplotlib.pyplot as plt
import pyMuTrans as pm

import matlab
import matlab.engine

sc.settings.set_figure_params(dpi=100, frameon=False)

datadir = "data_2025_ut/"
plotdir = "../plots/2025_ut/"

Current directory: /data/rds/DMP/DUDMP/PAEDCANC/echen/MuTrans-release-main/Example


In [2]:
for filename in os.listdir(datadir):
    file_path = os.path.join(datadir, filename)
    print(file_path)

data_2025_ut/grnb5_seurat.h5ad
data_2025_ut/nb039_seurat.h5ad
data_2025_ut/nb039_ut_scanpy.h5ad
data_2025_ut/nb039_ut_scanpy_nn.h5ad
data_2025_ut/nb067_seurat.h5ad
data_2025_ut/nb067_ut_scanpy.h5ad
data_2025_ut/nb067_ut_scanpy_nn.h5ad
data_2025_ut/shep_seurat.h5ad
data_2025_ut/shep_ut_scanpy.h5ad
data_2025_ut/shep_ut_scanpy_nn.h5ad
data_2025_ut/shsy5y_seurat.h5ad
data_2025_ut/shsy5y_ut_scanpy.h5ad
data_2025_ut/shsy5y_ut_scanpy_nn.h5ad
data_2025_ut/sknsh_seurat.h5ad
data_2025_ut/sknsh_ut_scanpy.h5ad
data_2025_ut/sknsh_ut_scanpy_nn.h5ad


In [16]:
gc.collect()

84318

In [ ]:
filename = "grnb5_seurat.h5ad"
file_path = os.path.join(datadir, filename)
print(file_path)
sample_name = filename.replace("_seurat.h5ad", "")
print(f"Processing {filename}...")
        
adata = sc.read_h5ad(file_path)
adata.obs = adata.obs[['Sample', 'Barcode', 'batch', 'description',
                       'Condition', 'Rec', 'seurat_clusters.0.4',
                       'MES.Sig', 'ADRN.Sig', 'AMT.Sig',
                       'AMT.score', 'AMT.state']]
adata.obsm = []
        
df = pd.DataFrame.sparse.from_spmatrix(adata.X)
print(f"Raw counts added for {sample_name}")
print(df.head(10))
        
print(f"Beginning preprocessing of {sample_name} AnnData object")
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars = ['mt'], percent_top = None, log1p = False, inplace = True)
        
print(f"Normalising and scaling {sample_name} data")
sc.pp.normalize_total(adata, target_sum = 1e4)
sc.pp.log1p(adata)
        
sc.pp.highly_variable_genes(adata, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
sc.pl.highly_variable_genes(adata, show = False)
plt.savefig(f"{plotdir}{sample_name}_hvgs.png")
plt.close()       

adata.raw = adata
adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value = 10)
        
print(f"Reducing dimensions on {sample_name}")
sc.tl.pca(adata, svd_solver = 'arpack')
sc.pl.pca(adata, color = ['PHOX2B','PRRX1'], show = False)
plt.savefig(f"{plotdir}{sample_name}_pca_phox2b_prrx1.png")
plt.close()
sc.pl.pca_variance_ratio(adata, log = True, show = False)
plt.savefig(f"{plotdir}{sample_name}_pca_variance.png")
plt.close() 

sc.pp.neighbors(adata, n_neighbors = 10, n_pcs = 40)
sc.tl.umap(adata)
sc.pl.umap(adata, color = ['PHOX2B', 'PRRX1'], show = False)
plt.savefig(f"{plotdir}{sample_name}_umap_phox2b_prrx1.png")
plt.close()

print(f"Clustering {sample_name} data")
sc.tl.leiden(adata, resolution = 0.2)
sc.pl.umap(adata, color = 'leiden', show = False)
plt.savefig(f"{plotdir}{sample_name}_umap_clusters.png")
plt.close()

adata.write_h5ad(
    f"{datadir}{sample_name}_ut_scanpy.h5ad",
    compression = hdf5plugin.FILTERS["zstd"]
)
print(f"Saved clustered {sample_name} data")
        
sc.tl.rank_genes_groups(adata, 'leiden', method = 'wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes = 25, sharey = False, show = False)
plt.savefig(f"{plotdir}{sample_name}_cluster_markers.png")
plt.close()

sc.tl.paga(adata, groups = 'leiden')
sc.pl.paga(adata, show = False)
plt.savefig(f"{plotdir}{sample_name}_paga.png")
plt.close()
sc.pl.paga(adata, color=['PHOX2B', 'PRRX1'], show = False)
plt.savefig(f"{plotdir}{sample_name}_paga_phox2b_prrx1.png")
plt.close()

print("Preparing data for MuTrans")
sc.tl.tsne(adata, n_pcs = 30)
sc.pp.neighbors(adata, metric = 'cosine', n_neighbors = 60, use_rep = 'X')
adata.write_h5ad(
    f"{datadir}{sample_name}_ut_scanpy_nn.h5ad",
    compression = hdf5plugin.FILTERS["zstd"]
)
print(f"Saved input {sample_name} data for MuTrans")

data_2025_ut/grnb5_seurat.h5ad
Processing grnb5_seurat.h5ad...
Raw counts added for grnb5
   0      1      2      3      4      5      6      7      8      9      ...  \
0    0.0    0.0    2.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0  ...   
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    2.0  ...   
2    0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
3    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    1.0  ...   
4    0.0    1.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0  ...   
6    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0  ...   
7    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
8    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
9    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   

   21999  22000  22001  22002

c:\Users\echen\Anaconda3\envs\mutrans\lib\site-packages\anndata\_core\anndata.py:1294: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
c:\Users\echen\Anaconda3\envs\mutrans\lib\site-packages\anndata\_core\anndata.py:1294: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
c:\Users\echen\Anaconda3\envs\mutrans\lib\site-packages\anndata\_core\anndata.py:1294: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


MemoryError: Unable to allocate 284. MiB for an array with shape (74347860,) and data type int32

: 